
# Python II - Basic imaging processing with python
<img src="Logo_python_nanocourse.PNG" width="250">

### Instructor: Paniz Karbasi

# Agenda

#### 0. Loading and visualising images
#### 1. Images are arrays of numbers or matrices: Basic matrix operations (transformations)
#### 2. Intensity enhancement: Histogram Equalization
#### 3. Segmentation: Adaptive thresholding, Distance transformation, Detection of maxima, Watershed
#### 4. Analysis: Extracting measurements from segmentation
#### 5. Advanced topics: PCA and K-means clustering (Take-Home Guided Practice )

### 0. Loading and visualising images

In [ ]:
# Set the input directory where the image files exist
# Windows example: 'C:/Users/Paniz/Downloads/'
base_dir = '/home2/s183990/jupyter_notebooks/'
input_dir = base_dir + 'pythonII-imaging/images/'

In [ ]:
# Make a list of files in the directory
from os import listdir
filelist = listdir(input_dir)
# print the file list
filelist

In [ ]:
# Sort the filelist 
filelist.sort()
filelist

In [ ]:
# Collect the file names only for files that are tifs: Classical solution
image_names = []
for filename in filelist:
    if filename[-4:]=='.tif':
        image_names.append(filename)

In [ ]:
# Collect the file names only for files that are tifs: Elegant solution
image_names = [filename for filename in filelist if filename[-4:]=='.tif'] 

In [ ]:
# Check that you have the right files in tiflist. 
print('Found', len(image_names), 'tif files in ', input_dir, ': ')
for file in image_names:
    print(file)

In [ ]:
# Read the images and store into a list
import skimage.io as io # Image file manipulation module
import os #
image_data = [io.imread(os.path.join(input_dir,image)) for image in image_names]

In [ ]:
# Get the shape of each image in the image_data
for image in image_data:
    print('Image shape: ', image.shape)

In [ ]:
# Slicing: Extracting certain slices of an image
image = image_data[0][1,:,:]

In [ ]:
image

In [ ]:
# Visualize the images
import matplotlib.pyplot as plt

for im_data,im_name,i in zip(image_data, image_names, range(1,len(image_data)+1)):
    plt.subplot(1, 2, i)
    plt.title(im_name)
    plt.imshow(im_data[1,:,:], cmap='gray')

plt.show()

### 1. Images are arrays of numbers or matrices: Basic matrix operations: Slicing, Filtering

In [ ]:
#1.0 Slicing
image_a = image_data[0][1,:,:]

In [ ]:
image_a[10:13, 20:23]

In [ ]:
mask = image_a < 87
mask

In [ ]:
# 1.1 Filtering
import numpy as np
image = image_data[0][1,:,:]
from skimage import img_as_float
image = img_as_float(image)
noise = np.random.normal(0, 0.09, image.shape)
image_noise = image + noise

In [ ]:
plt.subplot(1, 2, 1)
plt.title('Original image')
plt.imshow(image, cmap='gray')

plt.subplot(1, 2, 2)
plt.title('Noisy image')
plt.imshow(image_noise, cmap='gray')

plt.show()

In [ ]:
from scipy import ndimage
gauss_denoised = ndimage.gaussian_filter(image_noise, 2)
med_denoised = ndimage.median_filter(image_noise, 2)

In [ ]:
plt.subplot(1, 2, 1)
plt.title('Gauss denoised')
plt.imshow(gauss_denoised, cmap='gray')

plt.subplot(1, 2, 2)
plt.title('Med denoised')
plt.imshow(med_denoised, cmap='gray')

plt.show()

In [ ]:
# 1.2 Morphological filtering
# Probe an image with a simple shape (a structuring element), and modify 
# this image according to how the shape locally fits or misses the image
el = ndimage.generate_binary_structure(2, 1)
el

In [ ]:
plt.imshow(el, cmap=plt.cm.gray)
plt.title('Structuring element')
plt.show()

In [ ]:
el.astype(np.int)

In [ ]:
# Erosion = minimum filter. 
a = np.zeros((7,7), dtype=np.int)
a[1:6, 2:5] = 1
a

In [ ]:
plt.imshow(a, cmap='gray')
plt.title('Image')
plt.show()

In [ ]:
# Apply erosion and display the result
a_e = ndimage.binary_erosion(a).astype(a.dtype)
plt.imshow(a_e, cmap='gray')
plt.title('Erosion')
plt.show()

In [ ]:
a = np.zeros((5, 5))
a[2, 2] = 1
plt.imshow(a, cmap='gray')
plt.title('Image')
plt.show()

In [ ]:
# Apply the dilation and display the results
a_d = ndimage.binary_dilation(a).astype(a.dtype)
plt.imshow(a_d, cmap='gray')
plt.title('Dilation')
plt.show()

In [ ]:
a = np.zeros((5,5), dtype=np.int)
a[1:4, 1:4] = 1 ; a[4, 4] = 1
plt.imshow(a, cmap='gray')
plt.title('Image')
plt.show()

In [ ]:
# Apply opening and display the result
a_o = ndimage.binary_opening(a, structure=np.ones((3,3))).astype(np.int)
plt.imshow(a_o, cmap='gray')
plt.title('Opening')
plt.show()

In [ ]:
a = np.zeros((5,5), dtype=np.int)
a[1:4,1:4] = 1; a[2,2] = 0
plt.imshow(a, cmap='gray')
plt.title('Image')
plt.show()

In [ ]:
# Apply closing and display the result
a_o = ndimage.binary_closing(a, structure=np.ones((3,3))).astype(np.int)
plt.imshow(a_o, cmap='gray')
plt.title('Closing')
plt.show()

### 2. Intensity enhancement: Histogram Equalization

In [ ]:
# Plotting the histogram of an image
image = image_data[0][1,:,:]
plt.hist(image.ravel(), bins=256, histtype='step', color='black')
plt.show()

In [ ]:
help(plt.hist)

### 3. Segmentation

#### 3.1. PREPROCESSING AND SIMPLE CELL SEGMENTATION:  
##### (I) SMOOTHING AND (II) ADAPTIVE THRESHOLDING

In [ ]:
# Slicing: We only work on one channel for segmentation
image = image_data[0][1,:,:]

In [ ]:
# Gaussian smoothing
import scipy.ndimage as ndi # Image processing package
sigma = 3                                                # Smoothing factor for Gaussian
image_smooth = ndi.filters.gaussian_filter(image,sigma) # Perform smoothing

In [ ]:
# Create an adaptive background
import numpy as np # Array manipulation package
struct = ((np.mgrid[:31,:31][0] - 15)**2 + (np.mgrid[:31,:31][1] - 15)**2) <= 15**2  # Create a disk-shaped structural element
from skimage.filters import rank            # Import module containing mean filter function
bg = rank.mean(image_smooth, selem=struct) # Run a mean filter over the image using the disc

In [ ]:
# Threshold using created background
image_mem = image_smooth >= bg

In [ ]:
# Visualize the image
plt.imshow(image_mem, cmap='gray')
plt.title('Thresholding segmentation')
plt.show()

In [ ]:
# Clean by morphological hole filling
image_mem = ndi.binary_fill_holes(np.logical_not(image_mem))

In [ ]:
plt.imshow(image_mem, cmap='gray')
plt.title('Thresholding segmentation after cleaning')
plt.show()

#### 3.2. IMPROVED CELL SEGMENTATION BY SEEDING AND EXPANSION:
##### (I) SEEDING BY DISTANCE TRANSFORM and (II) EXPANSION BY WATERSHED

In [ ]:
# Distance transform on thresholded membranes
image_dt= ndi.distance_transform_edt(image_mem)

In [ ]:
# Dilating (maximum filter) of distance transform improves results
image_dt = ndi.filters.maximum_filter(image_dt,size=10) 

In [ ]:
# Retrieve and label the local maxima
from skimage.feature import peak_local_max
image_max = peak_local_max(image_dt,indices=False,min_distance=10)  # Local maximum detection
image_max = ndi.label(image_max)[0] # Labeling

In [ ]:
# Get the watershed function and run it
from skimage.morphology import watershed
image_ws = watershed(image_smooth,image_max)

In [ ]:
plt.imshow(image_ws, cmap='gray')
plt.title('Watershed segmentation')
plt.show()

In [ ]:
# IDENTIFICATION OF CELL EDGES
    
# Define the edge detection function
def edge_finder(footprint_values):
    if (footprint_values == footprint_values[0]).all():
        return 0
    else:
        return 1
        
# Iterate the edge finder over the segmentation
image_edges = ndi.filters.generic_filter(image_ws,edge_finder,size=3)

In [ ]:
# POSTPROCESSING: REMOVING CELLS AT THE IMAGE BORDER    
    
# Create a mask for the image boundary pixels
boundary_mask = np.ones_like(image_ws)   # Initialize with all ones
boundary_mask[1:-1,1:-1] = 0 # Set middle square to 0

In [ ]:
# Iterate over all cells in the segmentation
current_label = 1
for cell_id in np.unique(image_ws):
    
    # If the current cell touches the boundary, remove it
    if np.sum((image_ws==cell_id)*boundary_mask) != 0:
        image_ws[image_ws==cell_id] = 0
            
        # This is to keep the labeling continuous, which is cleaner
    else:
        image_ws[image_ws==cell_id] = current_label
        current_label += 1

In [ ]:
plt.imshow(image_ws, cmap='gray')
plt.title('Watershed segmentation after cleaning')
plt.show()

In [ ]:
# MEASUREMENTS: SINGLE-CELL AND MEMBRANE READOUTS    
    
# Initialize a dict for results of choice
results = {"cell_id":[], "mean":[],"membrane_mean":[],"cell_size":[],"cell_outline":[]}
    
# Iterate over segmented cells
for cell_id in np.unique(image_ws)[1:]:
        
    # Mask the pixels of the current cell
    cell_mask = image_ws==cell_id  
    edge_mask = np.logical_and(cell_mask,image_edges)
        
    # Get the current cell's values
    # Note that the original raw data is used for quantification!
    results["cell_id"].append(cell_id)
    results["mean"].append(np.mean(image[cell_mask]))    
    results["membrane_mean"].append(np.mean(image[edge_mask]))    
    results["cell_size"].append(np.sum(cell_mask))
    results["cell_outline"].append(np.sum(edge_mask))

In [ ]:
# Scatter plot of red membrane mean intensity over cell size.

# Prepare colormap to color cells from each image differently
colors = plt.cm.jet(np.linspace(1,0,1))

plt.scatter(results["cell_size"],results["membrane_mean"],color=colors)
plt.xlabel("cell size")
plt.ylabel("membrane mean")

# Show or save result
plt.savefig('cells_scatter.png', bbox_inches='tight')
plt.show()

### 4. Analysis: Extracting measurements from segmentation

In [ ]:
# Some frequently used variables
labels = np.unique(image_ws)[1:]  # Labels of cells in segmentation
N = len(labels) # Number of cells in segmentation

In [ ]:
# Initialize feature space
fspace = np.zeros((N,3),dtype=np.float)

In [ ]:
# Get the centroids
cen = np.zeros((N,2))
for index,label in enumerate(labels):
    cen[index,:] = ndi.measurements.center_of_mass(image_ws==label)

In [ ]:
### CELL SIZE (as approximated by segmentation area)

for index,label in enumerate(labels):  
    # Count the number of pixels that are part of that object
    fspace[index,0] = np.sum(image_ws==label)

In [ ]:
### STANDARD DEVIATION OF MEMBRANE INTENSITY

for index,label in enumerate(labels): 
    # Dilation provides yet another way of detecting edge pixels
    edge = ndi.maximum_filter(image_ws!=label,size=3)
    edge[image_ws!=label] = 0
    
    # Stdev of intensity at the edge
    fspace[index,1] = np.std(image[edge==1])

In [ ]:
### INTENSITY ASYMMETRY as the distance of the intensity-weighted center of
#   mass from the unweighted centroid

from scipy.spatial.distance import pdist

for index,label in enumerate(labels):
    
    # Intensity-weighted center of mass
    int_cen = ndi.measurements.center_of_mass(np.ma.array(image,mask=image_ws!=label))
    
    # Distance between the two centroids
    # Note: Below I use the pairwise distance function pdist from scipy's
    #       spatial package. This function is useful for a lot of things in
    #       data analsis, but to be more explicit one could also calculate the
    #       distance between two centroids using simple Pythagoras:
    #           diff = int_cen - cen[index,:]
    #           dist = np.sqrt(np.square(diff[0]) + np.square(diff[1]))
    
    fspace[index,2] = pdist([int_cen,cen[index,:]])[0]

In [ ]:
# Visualize the fspace
plt.boxplot(fspace,labels=["CELL SIZE","STDEV OF MEM INT", "INT ASYM"])
plt.show()

### 5. Advanced topics: PCA and K-means clustering (Take-Home Guided Practice )

In [ ]:
### Performing PCA with scikit-learn
from sklearn.decomposition import PCA
pca_estimator = PCA()   # Instantiate estimator
pca_estimator.fit(fspace_w)      # Fit data
print("PC-explained variance:", pca_estimator.explained_variance_ratio_)  # Retrieve information
fspace_pca = pca_estimator.transform(fspace_w)                           # Transform data to PC space

### Some plotting

# Scatterplot of pc1 vs pc2
plt.scatter(fspace_pca[:,0],fspace_pca[:,1])
plt.xlabel("PC1")
plt.ylabel("PC2")
plt.show()

In [ ]:
# Scatterplots of pc1 vs the features
# Reveals that features 2 (STDEV OF MEM INT) and 3 (INT ASYM) correlate well
# with PC1, hence indicating that PC1 characterizes the intensity distribution.
fig,axx = plt.subplots(1,3)
axx[0].scatter(fspace_w[:,0],fspace_pca[:,0])
axx[0].set_title("PC1 vs Feature 1")
axx[1].scatter(fspace_w[:,1],fspace_pca[:,0]) 
axx[1].set_title("PC1 vs Feature 2")
axx[2].scatter(fspace_w[:,2],fspace_pca[:,0]) 
axx[2].set_title("PC1 vs Feature 3")
plt.show()

In [ ]:
# Perform k-means clustering on your data. To do so, you have to assume the
# number of clusters. To begin with, just try it with 3 clusters. Try doing the
# PCA for raw, normalized and PCA-transformed data to see the difference. Don't 
# forget to visualize your result.


### K-means clustering using scipy

# Perform the actual clustering
k = 3
import scipy.cluster.vq as svq
k_centroids,k_labels = svq.kmeans2(fspace_pca,k)

# Visualization in PC space
plt.scatter(fspace_pca[:,0],fspace_pca[:,1],c=k_labels,s=30) # Color coded data
plt.scatter(k_centroids[:,0],k_centroids[:,1],s=150,c=range(k),marker='*',) # Centroids
plt.xlabel("PC1")
plt.ylabel("PC2")
plt.show()

# Transparent overlay on the image
mapped = np.zeros_like(image,dtype=np.float)
for index,label in enumerate(labels):
    mapped[image_ws==label] = (k_labels[index] + 1) / np.max(k_labels+1) * 255
    
plt.imshow(image,cmap='gray',interpolation='none')
plt.imshow(mapped,interpolation='none',alpha=0.7) 
plt.show()